# Purpose
The purpose of this notebook is to investigate the relationship between the order of the three image channels when reading using ```tiff.imread``` vs. ```image.read(window)```.  These are the two methods for reading image data
used in the "Tile Overlapping Window" and "Tilewise Predict..." notebooks.

It appears in the last cell in this notebook that the two methods are equivalent.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import tifffile
import rasterio
from rasterio.windows import Window

In [ ]:
tiff_filename = "/kaggle/input/hubmap-kidney-segmentation/test/b2dc8411c.tiff"

imread_image = tifffile.imread( tiff_filename )
print( "imread_image.shape", imread_image.shape )  # (14844, 31262, 3)
print( "imread_image.dtype", imread_image.dtype )

window_row0 = 7000
window_col0 = 15000
window_row1 = 7005
window_col1 = 15008

imread_window_0 = imread_image[ window_row0 : window_row1,
                                window_col0 : window_col1,
                                0 ]
imread_window_1 = imread_image[ window_row0 : window_row1,
                                window_col0 : window_col1,
                                1 ]
imread_window_2 = imread_image[ window_row0 : window_row1,
                                window_col0 : window_col1,
                                2 ]

print( "imread_window_0:\n", imread_window_0 )
print( "imread_window_1:\n", imread_window_1 )
print( "imread_window_2:\n", imread_window_2 )

del imread_image

with rasterio.open( tiff_filename ) as image_dataset:
    rasterio_image = image_dataset.read( [ 1, 2, 3 ], 
                                        window = Window.from_slices( ( window_row0, window_row1 ),
                                                                     ( window_col0, window_col1 ) ) )
    rasterio_image = np.moveaxis( rasterio_image, 0, -1 )
    print( "rasterio_image.shape", rasterio_image.shape )
    print( "rasterio_image.dtype", rasterio_image.dtype )
    rasterio_window_0 = rasterio_image[ :, :, 0 ]
    rasterio_window_1 = rasterio_image[ :, :, 1 ]
    rasterio_window_2 = rasterio_image[ :, :, 2 ]
    
    del rasterio_image

    print( "rasterio_window_0:\n", rasterio_window_0 )
    print( "rasterio_window_1:\n", rasterio_window_1 )
    print( "rasterio_window_2:\n", rasterio_window_2 )
    
    assert np.all( imread_window_0 == rasterio_window_0 )
    assert np.all( imread_window_1 == rasterio_window_1 )
    assert np.all( imread_window_2 == rasterio_window_2 )

